In [1]:
# mobilemasr scraper


import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime

url_template = "https://mobilemasr.com/search?q=used+mobile&page={i}"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Create CSV file and write headers
with open('mobilemisr_phones.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['phone_name', 'Price', 'Seller_Name', 'Listing_URL', 'Scraped_Date'])

    for i in range(1, 21):  # Loop through pages 1 to 20
        try:
            print(f"Scraping page {i}...")
            
            url = url_template.format(i=i)
            response = requests.get(url, headers=headers, timeout=10)
            
            if response.status_code != 200:
                print(f"Failed to fetch page {i}. Status code: {response.status_code}")
                continue
                
            soup = BeautifulSoup(response.content, 'html.parser')
            phones = soup.find_all('div', class_="product-card max-w-sm bg-white rounded-lg relative font-noto-or-Trebuchet h-[409px] group w-[140px] sm:w-[180px] md:w-[200px] my-2")


            for phone in phones:
                try:
                    # Extract phone_name
                    phone_name_element = phone.find('div', class_="h-[47px] p-0 m-0")
                    phone_name = phone_name_element.text.strip() if phone_name_element else "No phone_name"

                    # Extract price
                    price_element = phone.find('p', class_="flex justify-between text-sm sm:text-base py-1 font-black")
                    price = price_element.text.strip() if price_element else "No price"

                    # Extract seller name
                    seller_element = phone.find('p', class_="text-xs font-[400] px-1")
                    seller_name = seller_element.text.strip() if seller_element else "No Seller Name"

                    # Extract listing URL
                    listing_url_element = phone.find('a')
                    if listing_url_element and listing_url_element.get('href'):
                        relative_url = listing_url_element['href']
                        listing_url = "https://mobilemasr.com" + relative_url
                    else:
                        listing_url = "No listing url"

                    # Get current date
                    scraped_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    
                    # Write data to CSV
                    writer.writerow([phone_name, price, seller_name, listing_url, scraped_date])
                    
                except Exception as e:
                    print(f"Error processing a phone on page {i}: {e}")
                    continue
                    
        except Exception as e:
            print(f"Error fetching page {i}: {e}")
            continue

print("Scraping completed! Data saved to mobilemisr_phones.csv")

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...
Scraping completed! Data saved to mobilemisr_phones.csv


In [2]:
# noon scraper


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import csv
from datetime import datetime
import time

# Set up the driver
driver = webdriver.Chrome()
driver.maximize_window()

# Create CSV file and write headers
with open('noon_used_phones.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['phone_name', 'Price', 'Seller_Name', 'Listing_URL', 'Scraped_Date'])

    try:
        # Navigate to Noon website
        print("Opening Noon website...")
        driver.get("https://www.noon.com/")
        
        # Wait for search box and search for "used phones"
        wait = WebDriverWait(driver, 15)
        search_box = wait.until(EC.element_to_be_clickable((By.ID, "search-input")))
        
        print("Searching for 'used phones'...")
        search_box.clear()
        search_box.send_keys("used phones")
        search_box.send_keys(Keys.RETURN)
        
        # Wait for search results to load
        print("Waiting for search results...")
        time.sleep(5)
        
        # Scroll to load more products
        print("Loading more products...")
        for _ in range(3):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
        
        # Get the page source after JavaScript has loaded everything
        page_source = driver.page_source
        
        # Parse with BeautifulSoup
        soup = BeautifulSoup(page_source, 'html.parser')
        
        # Find all product containers
        products = soup.find_all('div' , class_ ="PBoxLinkHandler_linkWrapper__8FlHB")
        
        print(f"Found {len(products)} products")
        
        for product in products:
            try:
                # Extract phone name using title attribute
                try:
                    name_element = product.find('h2', class_ ="ProductDetailsSection_title__JorAV")
                    phone_name = name_element.get('title', 'No phone name')
                except (AttributeError, TypeError):
                    phone_name = "No phone name"
                                            
                # Extract price
                try:
                    price_element = product.find('div' , class_ = "Price_sellingPrice__HFKZf")
                    price = price_element.get_text(strip=True) if price_element else "No price"
                except (AttributeError, TypeError):
                    price = "No price"
                
                # Extract seller/rating
                try:
                    rating_element = product.find('div' , class_ = "RatingPreviewStar_textCtr__sfsJG")
                    seller_name = rating_element.get_text(strip=True) if rating_element else "No Rating"
                except (AttributeError, TypeError):
                    seller_name = "No Rating"
                
                # Extract listing URL
                try:
                    link_element = product.find('a')
                    listing_url = link_element.get('href')
                    if listing_url and listing_url != 'No listing url' and not listing_url.startswith('http'):
                        listing_url = "https://www.noon.com" + listing_url
                except (AttributeError, TypeError):
                    listing_url = "No listing url"
                
                # Get current date
                scraped_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                
                # Write data to CSV
                writer.writerow([phone_name, price, seller_name, listing_url, scraped_date])
                print(f"Scraped: {phone_name}")
                
            except Exception as e:
                print(f"Error processing a product: {e}")
                continue
                
    except Exception as e:
        print(f"An error occurred: {str(e)}")
    
    finally:
        # Close the driver
        driver.quit()

print("Scraping completed! Data saved to noon_used_phones.csv")

Opening Noon website...
Searching for 'used phones'...
Waiting for search results...
Loading more products...
Found 15 products
Scraped: ابل  هاتف آيفون 13 بذاكرة داخلية 128 جيجابايت وتقنية 5G مع تطبيق فيس تايم بلون ميدنايت - إصدار عالمي
Scraped: ابل  آيفون 16 برو ماكس سعة 256 جيجابايت تيتانيوم صحراوي 5G مع فيس تايم - نسخة الشرق الأوسط
Scraped: سامسونج جالاكسي زد فليب 7 ثنائي الشريحة ظل أزرق 12 جيجابايت 256 جيجابايت 5G - نسخة الشرق الأوسط
Scraped: سامسونج جالاكسي S25 ألترا AI ثنائي الشريحة رمادي تيتانيوم 12 جيجابايت RAM 256 جيجابايت 5G - نسخة الشرق الأوسط
Scraped: سامسونج جالاكسي Z فليب 7 ثنائي الشريحة أسود جت 12 جيجابايت 256 جيجابايت 5G - نسخة الشرق الأوسط
Scraped: سامسونج Galaxy Z Flip 7 Dual SIM Jet Black 12GB 256GB 5G - International Version
Scraped: سامسونج Galaxy Z Flip 7 Dual SIM Jet Black 12GB 512GB 5G - International Version
Scraped: سامسونج هاتف Galaxy Z Fold 6 ثنائي الشريحة بلون فضي وذاكرة وصول عشوائي (RAM) سعة 12 جيجابايت وذاكرة تخزين داخلية 256 جيجابايت يدعم تقنية 5G مع هد